In [2]:
import cv2
import time
import math
import asyncio
import pyautogui
import numpy as np

from mss import mss
from enum import Enum

In [3]:
%run Pathfinding.ipynb
%run astar.ipynb

In [4]:
def screenshot():
    field_width = 688
    field_height = 608
    field_left = 607
    field_top = 302
    
    with mss() as sct:        
        monitor_number = 1
        mon = sct.monitors[monitor_number]
        monitor = {'mon': monitor_number, 'width': field_width, 'height': field_height, 'left': mon["left"] + field_left, 'top': mon["top"] + field_top}

        sct_img = sct.grab(monitor)
        img = np.array(sct_img.pixels).astype(np.uint8)[::,::,::-1]
        
        return img

In [5]:
class Direction(Enum):
    TOP = 1
    LEFT = 2
    BOTTOM = 3
    RIGHT = 4

In [6]:
def calc_direction(current_pos, old_pos):
#     start_time = time.perf_counter()
    
    current_pos_no = 10 * current_pos.x + current_pos.y
    old_pos_no = 10 * old_pos.x + old_pos.y
    pos_diff = current_pos_no - old_pos_no
    
    ret_direction = None
    if(pos_diff == -1):
        ret_direction = Direction.TOP
    elif(pos_diff == -10):
        ret_direction = Direction.LEFT
    elif(pos_diff == 1):
        ret_direction = Direction.BOTTOM
    elif(pos_diff == 10):
        ret_direction = Direction.RIGHT
    
#     duration = time.perf_counter() - start_time
    
    return ret_direction

In [7]:
# detect game over
def game_over(img):
    (left, top, right, bottom) = (453, 436, 533, 515)
    img_crp = img[top:bottom, left:right]
    
    img_score_path = 'img_raw/gameover.png'
    img_score_full = cv2.imread(img_score_path)
    img_score = img_score_full[top:bottom, left:right]
    
    hist_crp = calc_hist(img_crp)
    hist_score = calc_hist(img_score)
    
    sum_diff = 0
    for chan in range(3):
        diff = cv2.compareHist(hist_crp[chan], hist_score[chan], cv2.HISTCMP_CORREL)
        sum_diff += diff
    
    avg_diff = sum_diff/3
    if(avg_diff > 0.9):
        return True
    
    return False

In [8]:
def countdown():
    time.sleep(1)
    print('3')
    time.sleep(1)
    print('2')
    time.sleep(1)
    print('1')

In [9]:
# find the optimal path and control the snake to move along the path
async def pathfinding(cells, head_pos, apple_pos, snake_speed):    
    if(head_pos is None):
        print('None 1')
        return None
    
    cm_pos = center_of_mass(cells, nrows, ncols)
#     min_body_pos = find_min_max_body_pos(cells, nrows, ncols, True)
#     max_body_pos = find_min_max_body_pos(cells, nrows, ncols, False)

    new_cells, path = find_optimal_path(cells, head_pos, [cm_pos], [1])
    if(new_cells is None):
        print('None 2')
        return None
    
    print(new_cells)        
    print("=============================")
    
    task = asyncio.create_task(control_snake(path, apple_pos, snake_speed))
    await task
    
    return task.result()[0], task.result()[1]

In [21]:
async def control_snake(path, apple_pos, snake_speed):
    init_direction = None    
    current_direction = init_direction
    old_pos = path[0]
    
    for i, pos in enumerate(path[1:]):
        direction = calc_direction(pos, old_pos)    # snake's turn direction
        old_pos = pos
        
        if(not direction is None):
            if(direction == Direction.TOP):
                # press up arrow
                pyautogui.press('up')
                print('up')
            if(direction == Direction.LEFT):
                # press left arrow
                pyautogui.press('left')
                print('left')
            if(direction == Direction.BOTTOM):
                # press down arrow
                pyautogui.press('down')
                print('down')
            if(direction == Direction.RIGHT):
                # press right arrow
                pyautogui.press('right')
                print('right')
            
            if(i==len(path)-2):
                await asyncio.sleep(snake_speed - 0.05)
                break
            await asyncio.sleep(snake_speed)
    
    near_wall, near_wall_direction = turn_if_near_wall(direction, apple_pos)
    
    return near_wall, near_wall_direction

In [11]:
def turn_if_near_wall(direction, head_pos):
    left_wall_x = 0
    right_wall_x = 16
    top_wall_y = 0
    bottom_wall_y = 14
    
    dx1 = head_pos.x - left_wall_x
    dx2 = right_wall_x - head_pos.x
    dy1 = head_pos.y - top_wall_y
    dy2 = bottom_wall_y - head_pos.y
    
    delta_dx = dx1 - dx2
    delta_dy = dy1 - dy2
    
    k = 5    # space from head to wall threshold
    if((direction == Direction.LEFT and dx1 <= k) or
       (direction == Direction.RIGHT and dx2 <= k)):
        if(delta_dy > 0):          # dy1 > dy2
            pyautogui.press('up')
            print('up')
            return True, Direction.TOP
        else:
            pyautogui.press('down')
            print('down')
            return True, Direction.BOTTOM
        
    if((direction == Direction.TOP and dy1 <= k) or
       (direction == Direction.BOTTOM and dy2 <= k)):
        if(delta_dx > 0):          # dx1 > dx2
            pyautogui.press('left')
            print('left')
            return True, Direction.LEFT
        else:
            pyautogui.press('right')
            print('right')
            return True, Direction.RIGHT
    
    return False, direction

In [12]:
def augment_snake_body(cells, head_pos, direction, n_augment):
    if(direction == Direction.TOP):
        for i in range(n_augment):
            cells[head_pos.y - i, head_pos.x] = Label.BODY.value
        new_head_pos = Position(head_pos.y - n_augment, head_pos.x)
        
    if(direction == Direction.BOTTOM):
        for i in range(n_augment):
            cells[head_pos.y + i, head_pos.x] = Label.BODY.value
        new_head_pos = Position(head_pos.y + n_augment, head_pos.x)
            
    if(direction == Direction.LEFT):
        for i in range(n_augment):
            cells[head_pos.y, head_pos.x - i] = Label.BODY.value
        new_head_pos = Position(head_pos.y, head_pos.x - n_augment)
            
    if(direction == Direction.RIGHT):
        for i in range(n_augment):
            cells[head_pos.y, head_pos.x + i] = Label.BODY.value
        new_head_pos = Position(head_pos.y, head_pos.x + n_augment)
        
    cells[new_head_pos.y, new_head_pos.x] = Label.HEAD.value
            
    return cells, new_head_pos

#  With concurrency

In [17]:
async def main():    
    pyautogui.PAUSE = 0.0
    pyautogui.FAILSAFE = False
    threshold = 0
    snake_speed = 0.1713
    apple_pos_cached = None
    first_move = True
    near_wall_direction = Direction.RIGHT
    
    countdown()
    
    while(True):
        start_time = time.perf_counter()
        img = screenshot()            
        cells, head_pos, apple_pos = get_cells(img, Label, threshold)
#         print(cells)
#         print("=============================")
            
        if(not apple_pos is None and (apple_pos_cached is None or apple_pos_cached != apple_pos)):
            if(first_move):
                first_move = False
            else:
                exec_time = time.perf_counter() - start_time + 0.02
                n_augment = exec_time / snake_speed
                print(n_augment)
                n_augment = round(n_augment)
                print(n_augment)
                cells, head_pos = augment_snake_body(cells, head_pos, near_wall_direction, n_augment)
            
            print(cells)
            print('Apple: ', apple_pos)
            print("=============================")
            near_wall, near_wall_direction = await pathfinding(cells, head_pos, apple_pos, snake_speed)
            apple_pos_cached = apple_pos 

        # check whether the game is over
        gameover = game_over(img)
        if(gameover):
            print("game over")
            break

In [22]:
await main()

3
2
1
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 2 3 0 0 0 0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
Apple:  (12,7)
[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  2  3 -1 -1 -1 -1 -1 -1 -1 -1  1  0  

down
down
down
down
right
2.7411290134267854
3
[[0 0 0 0 2 0 2 0 2 0 2 0 2 0 0 0 0]
 [0 0 0 0 0 2 2 2 0 2 0 2 0 0 0 0 0]
 [0 0 0 0 2 0 1 0 2 0 0 0 2 0 0 0 0]
 [0 0 0 0 0 2 0 2 0 2 0 2 0 0 0 0 0]
 [0 0 0 0 2 0 2 0 2 0 2 0 2 0 0 0 0]
 [0 0 0 0 0 2 0 2 0 2 0 2 0 0 0 0 0]
 [0 0 0 0 2 0 2 2 2 2 3 0 2 0 0 0 0]
 [0 0 0 0 2 2 2 2 2 2 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 2 2 2 2 2 2 2 2 2 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
Apple:  (6,2)


KeyError: <__main__.Position object at 0x0000017E50D05508>